In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymn
import anndata as ad
import time
import os
from pyprojroot import here
import resource

In [ ]:
here()
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [ ]:
#data below is from the Allen Institute

In [ ]:
adata_Zeng_v2 = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_10Xv2.h5ad") 

In [ ]:
adata_Zeng_v2.obs

In [ ]:
#replace the main log-normalized count matrix with the raw counts

In [ ]:
adata_Zeng_v2.X = adata_Zeng_v2.layers['rawcount']

In [ ]:
del adata_Zeng_v2.layers['rawcount']

In [ ]:
adata_Zeng_v3 = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_10Xv3.h5ad") 

In [ ]:
adata_Zeng_v3.X = adata_Zeng_v3.layers['rawcount']
del adata_Zeng_v3.layers['rawcount']

In [ ]:
adata_Zeng_v2.obs.shape

In [ ]:
adata_Zeng_v3.obs.shape

In [ ]:
adata_Zeng_v2.var.shape

In [ ]:
adata_Zeng_v3.var.shape

In [ ]:
adata_Zeng_v3.var

In [ ]:
#filter for Macosko ensmbl IDs to save space

In [ ]:
adata_macosko = sc.read_h5ad(base_data_folder + '/whole_mouse_brain/processed/macosko/20221007_atlas_500umis_mt-1pct.h5ad', backed = "r")
gene_map = adata_macosko.var


In [ ]:
Macosko_universe = gene_map['ensmblID'].to_list()

In [ ]:
adata_Zeng_v3.var["in_Macosko"] = adata_Zeng_v3.var.gene_identifier.isin(Macosko_universe)


In [ ]:
adata_Zeng_v2.var["in_Macosko"] = adata_Zeng_v2.var.gene_identifier.isin(Macosko_universe)


In [ ]:
adata_Zeng_v3 = adata_Zeng_v3[:, adata_Zeng_v3.var["in_Macosko"]]

In [ ]:
adata_Zeng_v2 = adata_Zeng_v2[:, adata_Zeng_v2.var["in_Macosko"]]

In [ ]:
#write out and reload for merge due to memory limits, should be smaller

In [ ]:
adata_Zeng_v2.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.v2.20k_genes_counts_only.h5ad")

In [ ]:
adata_Zeng_v3.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.v3.20k_genes_counts_only.h5ad")

In [ ]:
#############################
#restart the kernel to free memory
#############################

In [ ]:
adata_Zeng_v2 = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.v2.20k_genes_counts_only.h5ad") 

In [ ]:
adata_Zeng_v3 = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.v3.20k_genes_counts_only.h5ad") 

In [ ]:
adata_Zeng_multiome = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_multiome.h5ad")

In [ ]:
adata_Zeng_multiome.obs.groupby("roi").size().sort_values(ascending=False)

In [ ]:
adata_Zeng_multiome.X = adata_Zeng_multiome.layers['rawcount']
del adata_Zeng_multiome.layers['rawcount']

In [ ]:
adata_Zeng_multiome.obs.shape

In [ ]:
adata_Zeng_multiome.var

In [ ]:
#add in ensmbl ID's
adata_Zeng_multiome.var = adata_Zeng_multiome.var.merge(adata_Zeng_v3.var, left_index=True,right_index=True, how='left')

In [ ]:
adata_Zeng_multiome.shape

In [ ]:
adata_Zeng_multiome = adata_Zeng_multiome[:, adata_Zeng_multiome.var["in_Macosko"]==True]

In [ ]:
adata_Zeng_multiome.shape

In [ ]:
merged=ad.concat([adata_Zeng_v2, adata_Zeng_v3, adata_Zeng_multiome], join="inner")

In [ ]:
merged

In [ ]:
merged.var

In [ ]:
#seems the var data gets lost in merge, add it back
merged.var = adata_Zeng_v2.var

In [ ]:
merged.shape

In [ ]:
merged.obs.iloc[1,:]

In [ ]:
merged.obs['external_donor_name'] = merged.obs['external_donor_name'].astype(str)

In [ ]:
merged.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad")

In [ ]:
merged.obs

In [ ]:
print("Done!")